In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder

In [3]:
df_all = pd.read_csv('./all_donnees.csv')
df_alea = pd.read_csv('./donnees_aleatoires.csv')

In [4]:
le = LabelEncoder()
df_all['nom'] = le.fit_transform(df_all['nom'])
df_alea['nom'] = le.fit_transform(df_alea['nom'])

In [5]:
y_train= df_alea[['pourcentage_voix']].copy()
y_test = df_all[['pourcentage_voix']].copy()

X_train= df_alea[["part_pauvres_diff","part_modestes_diff","part_medians_diff","part_plutot_aises_diff",
                 "part_aises_diff","actemp","actcho","inactret","age_moyen","total_voix_par_ville","nom"
                 ,"insecurite","taux_chomage_moyen"]].copy()

X_test= df_all[["part_pauvres_diff","part_modestes_diff","part_medians_diff","part_plutot_aises_diff",
                 "part_aises_diff","actemp","actcho","inactret","age_moyen","total_voix_par_ville","nom"
                 ,"insecurite","taux_chomage_moyen"]].copy()

In [46]:
# Affichage de la dataset
print(X_train.head())
print(y_train.head())

   part_pauvres_diff  part_modestes_diff  part_medians_diff  \
0               12.4                25.6               28.1   
1               12.4                25.6               28.1   
2               12.4                25.6               28.1   
3               12.4                25.6               28.1   
4               12.4                25.6               28.1   

   part_plutot_aises_diff  part_aises_diff  actemp  actcho  inactret  \
0                    25.6              8.3    41.5     6.0      20.3   
1                    25.6              8.3    41.5     6.0      20.3   
2                    25.6              8.3    41.5     6.0      20.3   
3                    25.6              8.3    41.5     6.0      20.3   
4                    25.6              8.3    41.5     6.0      20.3   

   age_moyen  total_voix_par_ville  nom  insecurite  taux_chomage_moyen  
0       39.7                   599    0         0.0               7.375  
1       39.7                   599   10 

In [6]:
# Initialize the model
gbr = GradientBoostingRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0]
}

# Perform grid search with cross-validation
grid_search_gradient = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_gradient.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search_gradient.best_params_)

# Best model
best_model_gradient = grid_search_gradient.best_estimator_

# Evaluate the model on the test set
y_pred_gradient = best_model_gradient.predict(X_test)
mse = mean_squared_error(y_test, y_pred_gradient)
print(f"Test MSE: {mse}")

f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-

Best parameters found:  {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300, 'subsample': 0.8}
Test MSE: 12.147173666759876


In [7]:
r2 = r2_score(y_test, y_pred_gradient)
print(f"Test R2: {r2}")

Test R2: 0.8525894036185079


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import  KFold, GridSearchCV

In [62]:
from keras.saving import register_keras_serializable
@register_keras_serializable()
def r2_keras(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res / (SS_tot + K.epsilon()))


In [10]:
def build_model(layers=[64, 32], input_dim=len(X_train)):
    model = Sequential()
    model.add(Dense(layers[0], input_dim=input_dim, activation='relu'))
    for layer in layers[1:]:
        model.add(Dense(layer, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), r2_keras])
    return model


In [11]:
scaler = StandardScaler()
X_train_neural = scaler.fit_transform(X_train).copy()
X_test_neural = scaler.transform(X_test).copy()

In [12]:
from scikeras.wrappers import KerasRegressor

In [13]:
input_dim = X_train_neural.shape[1]
# Wrapper for the Keras model
model = KerasRegressor(build_fn=build_model, input_dim=input_dim, epochs=100, batch_size=10, verbose=0, layers=[1])

# Define the parameter grid
param_grid = {
    'layers': [[64, 32], [128, 64, 32], [128, 64, 32, 16]],
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

# Custom R2 scorer
r2_scorer = make_scorer(r2_score)

# Define the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=r2_scorer, cv=KFold(5))
grid_result = grid.fit(X_train_neural, y_train)


f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pa

In [14]:
# Best model parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

# Predict with the best model on the test set
best_model_neural = grid_result.best_estimator_
y_pred_neural = best_model_neural.predict(X_test_neural)

# Evaluate the best model
mse = mean_squared_error(y_test, y_pred_neural)
r2 = r2_score(y_test, y_pred_neural)
print(f"Test MSE: {mse}")
print(f"Test R2: {r2}")

Best: 0.8073057351093101 using {'batch_size': 20, 'epochs': 100, 'layers': [128, 64, 32]}
Test MSE: 11.81504368202964
Test R2: 0.8566199279584401


In [48]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
# Define the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('keras', best_model_neural),
        ('gb', best_model_gradient)
    ],
    final_estimator=LinearRegression()
)

# Train the stacking regressor
stacking_regressor.fit(X_train, y_train)

f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\ensemble\_stacking.py:967: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\scikeras\wrappers.py:925: UserW

StackingRegressor(estimators=[('keras',
                               KerasRegressor(batch_size=20, build_fn=<function build_model at 0x00000241C4A6B560>, epochs=100, input_dim=13, layers=[128, 64, 32], verbose=0)),
                              ('gb',
                               GradientBoostingRegressor(learning_rate=0.01,
                                                         max_depth=5,
                                                         min_samples_leaf=4,
                                                         n_estimators=300,
                                                         subsample=0.8))],
                  final_estimator=LinearRegression())

In [49]:
# Predict with the stacking regressor
stacking_predictions = stacking_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, stacking_predictions)
r2 = r2_score(y_test, stacking_predictions)

print(f"Stacking Model Test MSE: {mse}")
print(f"Stacking Model Test R2: {r2}")

Stacking Model Test MSE: 12.042234656352207
Stacking Model Test R2: 0.8538628786286009


In [63]:
import joblib

copy_model = stacking_regressor


# Save the stacking model
with open('stacking_model-temp.pkl', 'wb') as f:
    joblib.dump(copy_model, f)

# Load
with open('stacking_model-temp.pkl', 'rb') as f:
    loaded_model: StackingRegressor = joblib.load(f)
    loaded_prediction = loaded_model.predict(X_test)
    mse = mean_squared_error(y_test, loaded_prediction)
    r2 = r2_score(y_test, loaded_prediction)




In [64]:
print(f"Stacking Model Test MSE: {mse}")
print(f"Stacking Model Test R2: {r2}")

Stacking Model Test MSE: 12.042234656352207
Stacking Model Test R2: 0.8538628786286009


In [47]:
"""
import joblib
best_model_neural.model_.save('neural_network.keras')
joblib.dump(best_model_gradient, 'gradient_bossting.pkl')

joblib.dump(stacking_regressor, 'stacking_regressor.pkl')
"""

['stacking_regressor.pkl']

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_test_endoced_label = label_encoder.fit_transform(y_test)
y_train_endoced_label = label_encoder.fit_transform(y_train)

f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
mlp = MLPRegressor(max_iter=1000, random_state=42)

# Create a parameter grid for grid search
param_grid = {
    'hidden_layer_sizes': [(50,), (128, 64, 32), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
    'batch_size': [32, 64, 128],
    'early_stopping': [True, False],
} 
# Define the GridSearchCV
grid_search_second_neural = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=2)

# Perform the grid search
grid_result_second_neural = grid_search_second_neural.fit(X_train_neural, y_train)



Fitting 5 folds for each of 480 candidates, totalling 2400 fits


f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1624: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:

# Print the best parameters and best score
print(f"Best parameters found: {grid_result_second_neural.best_params_}")
print(f"Best cross-validation r2: {grid_result_second_neural.best_score_}")

# Predict on the test set using the best model
best_model_second_neural = grid_result_second_neural.best_estimator_
y_pred = best_model_second_neural.predict(X_test)


print( mean_squared_error(y_test, y_pred))
print(  r2_score(y_test, y_pred) )

Best parameters found: {'activation': 'relu', 'alpha': 0.05, 'batch_size': 32, 'early_stopping': True, 'hidden_layer_sizes': (128, 64, 32), 'learning_rate': 'adaptive', 'solver': 'sgd'}
Best cross-validation r2: 0.8362328160629181
22215796.21772723
-269596.18033062393


f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [40]:
#joblib.dump(best_model_second_neural, 'neural_network_full_sklearn.pkl')
joblib.dump(best_model_second_neural, 'neural_network_full_sklearn_regression.pkl')

['neural_network_full_sklearn_regression.pkl']

In [42]:
stacking_regressor_full_sklearn = StackingRegressor(
    estimators=[
        ('neural', best_model_second_neural),
        ('gb', best_model_gradient)
    ],
    final_estimator=LinearRegression()
)

# Train the stacking regressor
stacking_regressor_full_sklearn.fit(X_train_neural, y_train)

f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\ensemble\_stacking.py:967: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingRegressor(estimators=[('neural',
                               MLPRegressor(alpha=0.05, batch_size=32,
                                            early_stopping=True,
                                            hidden_layer_sizes=(128, 64, 32),
                                            learning_rate='adaptive',
                                            max_iter=1000, random_state=42,
                                            solver='sgd')),
                              ('gb',
                               GradientBoostingRegressor(learning_rate=0.01,
                                                         max_depth=5,
                                                         min_samples_leaf=4,
                                                         n_estimators=300,
                                                         subsample=0.8))],
                  final_estimator=LinearRegression())

In [44]:
joblib.dump(stacking_regressor_full_sklearn, 'stacking_regressor_full_sklearn.pkl')

['stacking_regressor_full_sklearn.pkl']

In [43]:
stacking_predictions_full_sklearn = stacking_regressor_full_sklearn.predict(X_test)

# Evaluate the model
mse_full_sklearn = mean_squared_error(y_test, stacking_predictions_full_sklearn)
r2_full_sklearn = r2_score(y_test, stacking_predictions_full_sklearn)

print(f"Stacking Model Test MSE: {mse_full_sklearn}")
print(f"Stacking Model Test R2: {r2_full_sklearn}")

f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
f:\Developpement\MSPR-Analyse-de-donnees\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


Stacking Model Test MSE: 2085142.4205019628
Stacking Model Test R2: -25303.00043490368


In [ ]:
param_grid = {
    'layers': [[32], [64, 32], [128, 64, 32], [128, 64, 32, 16], [256, 128, 64, 32]],
    'batch_size': [10, 20, 32, 64, 128, 256],
    'epochs': [50, 100, 150],
    'optimizer': ['SGD', 'Adam', 'RMSprop'],
}

# Custom R2 scorer
r2_scorer = make_scorer(r2_score)

# Define the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=r2_scorer, cv=KFold(5))
grid_result = grid.fit(X_train_neural, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

# Predict with the best model on the test set
best_model_neural = grid_result.best_estimator_
y_pred_neural = best_model_neural.predict(X_test_neural)

# Evaluate the best model
mse = mean_squared_error(y_test, y_pred_neural)
r2 = r2_score(y_test, y_pred_neural)
print(f"Test MSE: {mse}")
print(f"Test R2: {r2}")